In [1]:
import requests
import lxml.html as lh
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np


### Obtención de los jugadores seleccionables para el draft 2019

In [2]:
# Hay una noticia del 29/05/2019 donde se enumeran los jugadores universitarios que serán elegibles para el draft 2019. Hacemos scraping sobre esta lista para obtener los nombres de los jugadores

website_url = requests.get('https://collegebasketball.nbcsports.com/2019/05/29/2019-nba-draft-early-entry-list-who-declared-who-is-returning-who-are-we-waiting-on/')
soup = BeautifulSoup(website_url.content,'lxml')
soup.prettify()
name_list = soup.find('div',{'class':"entry-content"})
name_list
listdata = name_list.findAll('li')
data =[]
count = 0
for cell in listdata:
    count = count+1
    if (count >5):
        data.append(cell.get_text())

In [3]:
# Como el scraping me coge primero los que han rechazado ir al draft a pesar de confirmar inicialmente y los que irán o están en duda, buscamos el índice del primer jugador que va

first_confirmed_indx_player = data.index('NICKEIL ALEXANDER-WALKER, Virginia Tech')
display(first_confirmed_indx_player)
data = data[int(first_confirmed_indx_player):]


79

In [4]:
data2 = np.array(data)
players = [items for item in data2 for items in item.split(',')]
del players[1::2]

### Obtención de las estadisticas de los jugadores seleccionables

In [22]:
# La web de sports-reference tiene una pagina propia de cada jugador, esta tiene que llevar el nombre seguido de un "-1" o "-2" si hay otro con el mismo nombre. Convertimos los nombres a minuscula, quitamos caracteres especiales y cambiamos el espacio por una "-" para conseguir entrar en la página de cada jugador de la lista previa
# Con beautiful soup extraemo la información de la tabla y la guardamos en una variable
# Solo hacemos scraping de los jugadores que tienen estadísticas en la temporada 2018-2019

maindata = []
for name in players:
    realname = name
    name = name.replace("IGGY BRAZDEIKIS", "IGNAS-BRAZDEIKIS")
    name = name.replace("DEWAN HERNANDEZ", "DEWAN HUELL")
    name = name.replace("NIC CLAXTON", "NICOLAS-CLAXTON")
    name = name.replace("JA MORANT", "TEMETRIUS-MORANT")
    name = name.replace("KZ OKPALA", "KEZIE-OKPALA")
    name = name.replace("SIMI SHITTU", "SIMISOLA-SHITTU")
    name = name.replace("JA MORANT", "TEMETRIUSMORANT")
    name = name.lower()
    name = name.replace(" ", "-")
    name = name.replace(".", "")
    name = name.replace("\'", "")
    name = name.replace("’", "")
    name = name.replace("JR", "")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    My_table = soup.find('table',{'id':'players_per_game'})
    if My_table is not None:
        data2=[]
        tabledata2 = My_table.findAll('th')
        for cell in tabledata2:
            data2.append(cell.get_text())
        if '2018-19' in data2:
            maindata = maindata + [realname]
            data = []
            tabledata = My_table.findAll('td')
            for cell in tabledata:
                data.append(cell.get_text())
            maindata = maindata +data[-28:]
        else:
            print(name)

jared-harper
dewan-huell
sacha-killeya-jones
jacob-ledoux
trevor-manuel
jontay-porter
brandon-randolph
justin-simon


In [23]:
maindata

['NICKEIL ALEXANDER-WALKER',
 'Virginia Tech',
 '',
 '67',
 '67',
 '29.9',
 '4.7',
 '10.1',
 '.464',
 '3.0',
 '5.6',
 '.529',
 '1.7',
 '4.5',
 '.383',
 '2.4',
 '3.1',
 '.763',
 '0.6',
 '3.4',
 '4.0',
 '2.7',
 '1.4',
 '0.5',
 '2.2',
 '1.9',
 '13.5',
 '',
 '7.25',
 'R.J. BARRETT',
 'Duke',
 '',
 '38',
 '38',
 '35.3',
 '8.4',
 '18.5',
 '.454',
 '6.5',
 '12.2',
 '.529',
 '1.9',
 '6.2',
 '.308',
 '3.9',
 '5.9',
 '.665',
 '1.6',
 '6.0',
 '7.6',
 '4.3',
 '0.9',
 '0.4',
 '3.2',
 '1.8',
 '22.6',
 '',
 '11.97',
 'TYUS BATTLE',
 'Syracuse',
 '',
 '103',
 '94',
 '35.4',
 '5.3',
 '12.8',
 '.417',
 '3.6',
 '7.6',
 '.474',
 '1.7',
 '5.2',
 '.335',
 '3.6',
 '4.5',
 '.803',
 '0.5',
 '2.3',
 '2.8',
 '2.1',
 '1.3',
 '0.2',
 '1.8',
 '2.0',
 '16.0',
 '',
 '9.52',
 'BOL BOL',
 'Oregon',
 '',
 '9',
 '9',
 '29.8',
 '8.2',
 '14.7',
 '.561',
 '6.8',
 '11.9',
 '.570',
 '1.4',
 '2.8',
 '.520',
 '3.1',
 '4.1',
 '.757',
 '2.1',
 '7.4',
 '9.6',
 '1.0',
 '0.8',
 '2.7',
 '2.0',
 '1.7',
 '21.0',
 '',
 '6.13',
 'MARQUES

In [24]:
import numpy as np
maindata = np.array(maindata)
refined2 = np.reshape(maindata, (-1, 29))


In [25]:
# Extraemos los nombres de columna de la tabla

tableheader = My_table.findAll('th')
colheaders = []
count = 0
for item in tableheader:
    if (count < 29):
        colheaders.append(item.get_text())
        count=count+1
colheaders.remove('Season')
colheaders = ['Name'] + colheaders
print(colheaders)

['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', '\xa0', 'SOS']


In [26]:
# Convertimos la información a un DF con los nombres de las columnas establecidos

import pandas as pd
draft19_class = pd.DataFrame(refined2, columns=colheaders)
draft19_class.head()

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS
0,NICKEIL ALEXANDER-WALKER,Virginia Tech,,67,67,29.9,4.7,10.1,.464,3.0,...,3.4,4.0,2.7,1.4,0.5,2.2,1.9,13.5,,7.25
1,R.J. BARRETT,Duke,,38,38,35.3,8.4,18.5,.454,6.5,...,6.0,7.6,4.3,0.9,0.4,3.2,1.8,22.6,,11.97
2,TYUS BATTLE,Syracuse,,103,94,35.4,5.3,12.8,.417,3.6,...,2.3,2.8,2.1,1.3,0.2,1.8,2.0,16.0,,9.52
3,BOL BOL,Oregon,,9,9,29.8,8.2,14.7,.561,6.8,...,7.4,9.6,1.0,0.8,2.7,2.0,1.7,21.0,,6.13
4,MARQUES BOLDEN,Duke,,88,24,13.6,1.5,2.6,.573,1.5,...,1.9,3.3,0.4,0.3,1.1,0.6,1.6,3.8,,10.86


In [27]:
# Miramos la forma del DF

draft19_class.shape

(76, 29)

In [28]:
len(players)

94

In [29]:
# Vemos que jugadores de la lista no se ha podido conseguir la información por problemas con la URL
# Nota: los jugadores que faltan no se han podido encontrar en la web y otros no han sido incluídos por no tener estadisticas de 2019

a = list(draft19_class['Name'])
b = list(players)

faltan=[]

for player in b:
    if player not in a:
        faltan.append(player)
        
display(faltan)

['TROY BAXTER JR.',
 'DARIUS BAZLEY',
 'CHARLIE BROWN JR.',
 'JARED HARPER',
 'DEWAN HERNANDEZ',
 'AMIR HINTON',
 'DAULTON HOMMES',
 'SACHA KILLEYA-JONES',
 'JALEN LECQUE',
 'JACOB LEDOUX',
 'TREVOR MANUEL',
 'ZACH NORVELL JR.',
 'JONTAY PORTER',
 'KEVIN PORTER JR.',
 'BRANDON RANDOLPH',
 'AUSTIN ROBINSON',
 'JUSTIN SIMON',
 'JALEN SYKES']

### Obtención de la posición en el campo de los jugadores

In [30]:
# Por cada nombre de la tabla anterior extraemos la posición en la que juega el jugador

players = list(draft19_class['Name'])

pos = []
for name in players:
    realname = name
    name = name.replace("IGGY BRAZDEIKIS", "IGNAS-BRAZDEIKIS")
    name = name.replace("DEWAN HERNANDEZ", "DEWAN HUELL")
    name = name.replace("NIC CLAXTON", "NICOLAS-CLAXTON")
    name = name.replace("JA MORANT", "TEMETRIUS-MORANT")
    name = name.replace("KZ OKPALA", "KEZIE-OKPALA")
    name = name.replace("SIMI SHITTU", "SIMISOLA-SHITTU")
    name = name.replace("JA MORANT", "TEMETRIUSMORANT")
    name = name.lower()
    name = name.replace(" ", "-")
    name = name.replace(".", "")
    name = name.replace("\'", "")
    name = name.replace("’", "")
    name = name.replace("JR", "")
    website_url = requests.get('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
#    print('https://www.sports-reference.com/cbb/players/'+ name +'-1.html')
    soup = BeautifulSoup(website_url.content,'lxml')
    soup.prettify()
    position = soup.find_all('p')
    if position is not None:
        pos.append(list(position[0])[2].replace(" ", "").replace("\n", ""))

In [31]:
# Creamos solo posiciones de Guard, Forward y Center

position=[]
for e in pos:
    POS = e.split('-')
    position.append(POS[0])
    
draft19_class['POS'] = position

In [32]:
draft19_class.head()

,Name,School,Conf,G,GS,MP,FG,FGA,FG%,2P,...,TRB,AST,STL,BLK,TOV,PF,PTS,,SOS,POS
0,NICKEIL ALEXANDER-WALKER,Virginia Tech,,67,67,29.9,4.7,10.1,.464,3.0,...,4.0,2.7,1.4,0.5,2.2,1.9,13.5,,7.25,Guard
1,R.J. BARRETT,Duke,,38,38,35.3,8.4,18.5,.454,6.5,...,7.6,4.3,0.9,0.4,3.2,1.8,22.6,,11.97,Forward
2,TYUS BATTLE,Syracuse,,103,94,35.4,5.3,12.8,.417,3.6,...,2.8,2.1,1.3,0.2,1.8,2.0,16.0,,9.52,Guard
3,BOL BOL,Oregon,,9,9,29.8,8.2,14.7,.561,6.8,...,9.6,1.0,0.8,2.7,2.0,1.7,21.0,,6.13,Center
4,MARQUES BOLDEN,Duke,,88,24,13.6,1.5,2.6,.573,1.5,...,3.3,0.4,0.3,1.1,0.6,1.6,3.8,,10.86,Center


### Limpieza de columnas no usables en el modelo

In [33]:
draft19_class.columns

Index(['Name', 'School', 'Conf', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P',
       '2PA', '2P%', '3P', '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB',
       'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS', ' ', 'SOS', 'POS'],
      dtype='object')

In [34]:
remove_columns = ['School', 'Conf']

for i in remove_columns:
    draft19_class.drop([i], axis=1, inplace=True)

In [35]:
columns = ['Name', 'G', 'GS', 'MP', 'FG', 'FGA', 'FG%', '2P', '2PA', '2P%', '3P',
       '3PA', '3P%', 'FT', 'FTA', 'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS', 'x', 'SOS', 'POS']

draft19_class.columns = columns


In [36]:
draft19_class.drop(['x'], axis=1, inplace=True)

### Exportación de la tabla como .csv

In [39]:
# Salvamos el archivo

draft19_class.to_csv('../input/draft19_class.csv', index=False)